# Script to compile med data
- Drug types administered to cohort
- Drug specific dose distribution

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import matplotlib.pyplot
%matplotlib inline

In [2]:
import Pain_data_postgres_etl
pless_module = Pain_data_postgres_etl.PainMedETL()

## Subject List

In [3]:
cohort_df = pd.read_csv('cohort2334.csv', parse_dates=['starttime', 'endtime'])
cohort_df.shape

(2334, 13)

### Loop to get meds administered by type

In [4]:
def dn_totals(med_df):
    if type(med_df.index[0]) is not pd.tslib.Timestamp:
        med_df.set_index('Shift_Start', drop=False, inplace=True)
    col_list = med_df.columns
    med_day = med_df[med_df.index.hour == 7]
    med_night = med_df[med_df.index.hour == 19]
    med_totals = {}
    for col in col_list:
        if col != 'Shift_Start':
            key1 = 'D_'+ col
            med_totals[key1] = med_day[col].sum()
            key2 = 'N_'+col
            med_totals[key2] = med_night[col].sum()
    return med_totals        
            

Creating empty dataframe for output

In [5]:
med_list = pd.read_csv('pain_med_list.csv')
col_list = med_list['label'].tolist()

In [6]:
shift_col_list = ['Hadm_id']
for col in col_list:
    shift_col_list.append('D_'+col)
    shift_col_list.append('N_'+col)

In [7]:
cohort_bolus_totals = pd.DataFrame(columns= shift_col_list)
cohort_iv_totals = pd.DataFrame(columns= shift_col_list)
cohort_bolus_totals.head(2)

,Hadm_id,D_Lorazepam (Ativan),N_Lorazepam (Ativan),D_Diazepam (Valium),N_Diazepam (Valium),D_Midazolam (Versed),N_Midazolam (Versed),D_Ketamine,N_Ketamine,D_Fentanyl,...,D_Fentanyl (Push),N_Fentanyl (Push),D_Meperidine (Demerol),N_Meperidine (Demerol),D_Fentanyl (Concentrate),N_Fentanyl (Concentrate),D_Morphine Sulfate,N_Morphine Sulfate,D_Pentobarbital,N_Pentobarbital


In [12]:
#cohort_med_totals = pd.DataFrame(columns=['hadm_id', 'D_Lorazepam (Ativan)', 'NBZ', 'OP'])
for i, r in cohort_df.iterrows():
    if i > 10:
        break
    med_data = pless_module.get_med_data(r['hadm_id'])
    med_vent_data = pless_module.filter_by_duration(med_data, r['starttime'], r['endtime'])
    if med_vent_data.shape[0] > 0:
        med_bolus_df = med_vent_data.loc[med_vent_data['OrderCategoryName'] == '05-Med Bolus']
        med_iv_df = med_vent_data.loc[med_vent_data['OrderCategoryName'] == '01-Drips']
        if med_bolus_df.shape[0] > 0:
            bolus_shift_totals = pless_module.get_bolus_shift_total(med_bolus_df)
            totals_dict = dn_totals(bolus_shift_totals)
            cohort_bolus_totals['Hadm_id'].loc[i] = r['hadm_id']
            for key, value in totals_dict.iteritems():
                cohort_bolus_totals[key].loc[i] = value
            #bolus_save_file = 'Med_data\\Bolus_List_' + str(hadm_id) + '.csv'
            #bolus_shift_totals.to_csv(bolus_save_file)
            #if i%50 == 0:
                #print 'Saved bolus: '+ str(r['hadm_id'])
        if med_iv_df.shape[0] > 0:
            iv_shift_totals = pless_module.get_IV_shift_total(med_iv_df)
            if iv_shift_totals.shape[0] > 0:
                total_iv_dict = dn_totals(iv_shift_totals)
                cohort_iv_totals['Hadm_id'].loc[i] = r['hadm_id']
                for key, value in total_iv_dict.iteritems():
                    cohort_iv_totals[key].loc[i] = value
            #iv_save_file = 'Med_data\\IV_List_' + str(hadm_id) + '.csv'
            #iv_shift_totals.to_csv(iv_save_file)
            #if i%80 == 0:
                #print 'Saved IV: ' + str(r['hadm_id'])
    

In [13]:
print cohort_iv_totals.shape
totals_dict

(0, 27)


{u'D_Fentanyl': 2.5000000000000002e-08,
 u'D_Midazolam (Versed)': 0.00025000000000000001,
 u'D_Morphine Sulfate': 0.0050000004000000002,
 u'N_Fentanyl': nan,
 u'N_Midazolam (Versed)': nan,
 u'N_Morphine Sulfate': 0.015000001200000002}

In [16]:
cohort_bolus_totals.columns

Index([u'Hadm_id', u'D_Lorazepam (Ativan)', u'N_Lorazepam (Ativan)',
       u'D_Diazepam (Valium)', u'N_Diazepam (Valium)', u'D_Midazolam (Versed)',
       u'N_Midazolam (Versed)', u'D_Ketamine', u'N_Ketamine', u'D_Fentanyl',
       u'N_Fentanyl', u'D_Haloperidol (Haldol)', u'N_Haloperidol (Haldol)',
       u'D_Hydromorphone (Dilaudid)', u'N_Hydromorphone (Dilaudid)',
       u'D_Propofol', u'N_Propofol', u'D_Fentanyl (Push)',
       u'N_Fentanyl (Push)', u'D_Meperidine (Demerol)',
       u'N_Meperidine (Demerol)', u'D_Fentanyl (Concentrate)',
       u'N_Fentanyl (Concentrate)', u'D_Morphine Sulfate',
       u'N_Morphine Sulfate', u'D_Pentobarbital', u'N_Pentobarbital'],
      dtype='object')

In [21]:
for k, v in totals_dict.iteritems():
    print str(k)
    print v
    cohort_bolus_totals[k].loc[0] = v
type(cohort_bolus_totals[k])

D_Midazolam (Versed)
0.00025
D_Morphine Sulfate
0.0050000004
D_Fentanyl
2.5e-08
N_Fentanyl
nan
N_Morphine Sulfate
0.0150000012
N_Midazolam (Versed)
nan


pandas.core.series.Series

In [22]:
k

u'N_Midazolam (Versed)'

In [23]:
cohort_bolus_totals['D_Midazolam (Versed)'].head()

0    0.00025
1    0.01400
4    0.00750
5    0.01200
6    0.00500
Name: D_Midazolam (Versed), dtype: float64

In [24]:
cohort_bolus_totals['Hadm_id'].head()

0    165660
1    116367
2    100009
3    170157
4    102365
Name: Hadm_id, dtype: int64

In [25]:
t1 = pd.concat([cohort_bolus_totals['Hadm_id'], cohort_bolus_totals['D_Diazepam (Valium)'], 
                cohort_bolus_totals['D_Fentanyl']], axis = 1)

In [26]:
t1.head()

,Hadm_id,D_Diazepam (Valium),D_Fentanyl
0,165660,NaN,2.500000e-08
1,116367,0.05,5.500000e-07
2,100009,NaN,NaN
3,170157,NaN,1.500000e-07
4,102365,NaN,3.500000e-07


## Creating shift totals dataframe using concat

Bolus

In [27]:
bolus_shift_total = pd.concat([t1, cohort_bolus_totals['D_Fentanyl (Concentrate)'], cohort_bolus_totals['D_Fentanyl (Push)'],
                              cohort_bolus_totals['D_Haloperidol (Haldol)'], cohort_bolus_totals['D_Hydromorphone (Dilaudid)'],
                              cohort_bolus_totals['D_Ketamine'], cohort_bolus_totals['D_Lorazepam (Ativan)'],
                              cohort_bolus_totals['D_Meperidine (Demerol)'], cohort_bolus_totals['D_Midazolam (Versed)'],
                              cohort_bolus_totals['D_Morphine Sulfate'], cohort_bolus_totals['D_Pentobarbital'],
                              cohort_bolus_totals['D_Propofol'], cohort_bolus_totals['N_Diazepam (Valium)'],
                               cohort_bolus_totals['N_Fentanyl'], cohort_bolus_totals['N_Fentanyl (Concentrate)'],
                              cohort_bolus_totals['N_Fentanyl (Push)'], cohort_bolus_totals['N_Haloperidol (Haldol)'],
                              cohort_bolus_totals['N_Hydromorphone (Dilaudid)'], cohort_bolus_totals['N_Ketamine'],
                              cohort_bolus_totals['N_Lorazepam (Ativan)'], cohort_bolus_totals['N_Meperidine (Demerol)'],
                              cohort_bolus_totals['N_Midazolam (Versed)'], cohort_bolus_totals['N_Morphine Sulfate'],
                              cohort_bolus_totals['N_Pentobarbital'], cohort_bolus_totals['N_Propofol']], axis=1)
bolus_shift_total.head()


,Hadm_id,D_Diazepam (Valium),D_Fentanyl,D_Fentanyl (Concentrate),D_Fentanyl (Push),D_Haloperidol (Haldol),D_Hydromorphone (Dilaudid),D_Ketamine,D_Lorazepam (Ativan),D_Meperidine (Demerol),...,N_Fentanyl (Push),N_Haloperidol (Haldol),N_Hydromorphone (Dilaudid),N_Ketamine,N_Lorazepam (Ativan),N_Meperidine (Demerol),N_Midazolam (Versed),N_Morphine Sulfate,N_Pentobarbital,N_Propofol
0,165660,NaN,2.500000e-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.015,NaN,NaN
1,116367,0.05,5.500000e-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.007,NaN,NaN,NaN
2,100009,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.009,NaN,NaN
3,170157,NaN,1.500000e-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.05
4,102365,NaN,3.500000e-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.004,NaN,NaN,NaN


In [30]:
#Saving
bolus_shift_total.to_csv('Cohort_Shift_Bolus_Total.csv')

IV

In [10]:
cohort_iv_totals['Hadm_id'].head()

0    165660
1    116367
2    100009
3    170157
4    102365
Name: Hadm_id, dtype: int64

In [12]:
for i, r in cohort_df.iterrows():
    med_data = pless_module.get_med_data(r['hadm_id'])
    med_vent_data = pless_module.filter_by_duration(med_data, r['starttime'], r['endtime'])
    if med_vent_data.shape[0] > 0:
        #med_bolus_df = med_vent_data.loc[med_vent_data['OrderCategoryName'] == '05-Med Bolus']
        med_iv_df = med_vent_data.loc[med_vent_data['OrderCategoryName'] == '01-Drips']
        if med_iv_df.shape[0] > 0:
            iv_shift_totals = pless_module.get_IV_shift_total(med_iv_df)
            if iv_shift_totals.shape[0] > 0:
                total_iv_dict = dn_totals(iv_shift_totals)
                cohort_iv_totals['Hadm_id'].loc[i] = r['hadm_id']
                for key, value in total_iv_dict.iteritems():
                    cohort_iv_totals[key].loc[i] = value
    if i%100 == 0:
        print r['hadm_id']


165660
123253
103886
152964
125325
147997
137668
124365
186106
195066
129565
123064
173121
187978
136332
106308
161109
156068
144195
178012
102480
152763
173297
116420


In [13]:
t2 = pd.concat([cohort_iv_totals['Hadm_id'], cohort_iv_totals['D_Midazolam (Versed)'], 
                cohort_iv_totals['D_Fentanyl']], axis = 1)
t2.head()

,Hadm_id,D_Midazolam (Versed),D_Fentanyl
0,165660,28.144547,NaN
1,116367,217.508976,NaN
2,100009,NaN,NaN
3,170157,NaN,NaN
4,102365,13.229167,NaN


In [14]:
iv_shift_cohort = pd.concat([cohort_iv_totals['Hadm_id'], cohort_iv_totals['D_Diazepam (Valium)'],
                            cohort_iv_totals['D_Fentanyl'], cohort_iv_totals['D_Fentanyl (Concentrate)'],
                            cohort_iv_totals['D_Fentanyl (Push)'], cohort_iv_totals['D_Haloperidol (Haldol)'], 
                             cohort_iv_totals['D_Hydromorphone (Dilaudid)'], cohort_iv_totals['D_Ketamine'],
                             cohort_iv_totals['D_Lorazepam (Ativan)'], cohort_iv_totals['D_Meperidine (Demerol)'], 
                             cohort_iv_totals['D_Midazolam (Versed)'], cohort_iv_totals['D_Morphine Sulfate'], 
                             cohort_iv_totals['D_Pentobarbital'], cohort_iv_totals['D_Propofol'], 
                             cohort_iv_totals['N_Diazepam (Valium)'], cohort_iv_totals['N_Fentanyl'], 
                             cohort_iv_totals['N_Fentanyl (Concentrate)'], cohort_iv_totals['N_Fentanyl (Push)'], 
                             cohort_iv_totals['N_Haloperidol (Haldol)'], cohort_iv_totals['N_Hydromorphone (Dilaudid)'], 
                             cohort_iv_totals['N_Ketamine'], cohort_iv_totals['N_Lorazepam (Ativan)'], 
                             cohort_iv_totals['N_Meperidine (Demerol)'], cohort_iv_totals['N_Midazolam (Versed)'], 
                             cohort_iv_totals['N_Morphine Sulfate'], cohort_iv_totals['N_Pentobarbital'], 
                             cohort_iv_totals['N_Propofol']], axis = 1)
print iv_shift_cohort.shape
iv_shift_cohort.head()

(2181, 27)


,Hadm_id,D_Diazepam (Valium),D_Fentanyl,D_Fentanyl (Concentrate),D_Fentanyl (Push),D_Haloperidol (Haldol),D_Hydromorphone (Dilaudid),D_Ketamine,D_Lorazepam (Ativan),D_Meperidine (Demerol),...,N_Fentanyl (Push),N_Haloperidol (Haldol),N_Hydromorphone (Dilaudid),N_Ketamine,N_Lorazepam (Ativan),N_Meperidine (Demerol),N_Midazolam (Versed),N_Morphine Sulfate,N_Pentobarbital,N_Propofol
0,165660,NaN,NaN,2.026411,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,27.514534,NaN,NaN,NaN
1,116367,NaN,NaN,5.384718,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,299.665153,NaN,NaN,297.811063
2,100009,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.395842
3,170157,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,3.350000,NaN,NaN,319.387120
4,102365,NaN,NaN,1.733698,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,12.000000,NaN,NaN,NaN


In [15]:
#Saving
iv_shift_cohort.to_csv('Cohort_Shift_IV_Total.csv')

### Reading a bolus shift file

In [14]:
fname = 'Med_data/IV_List_194358.csv'
bolus_example = pd.read_csv(fname, parse_dates=['starttime'])
bolus_example

,starttime,Propofol
0,2132-01-01 07:00:00,12.167166
1,2132-01-01 19:00:00,69.860648
2,2132-01-02 07:00:00,100.266741
3,2132-01-02 19:00:00,32.313714


In [15]:
bolus_example['starttime'].iloc[0].hour

7

In [16]:
bolus_example.set_index('starttime', drop = False, inplace=True)
bolus_example

,starttime,Propofol
starttime,,
2132-01-01 07:00:00,2132-01-01 07:00:00,12.167166
2132-01-01 19:00:00,2132-01-01 19:00:00,69.860648
2132-01-02 07:00:00,2132-01-02 07:00:00,100.266741
2132-01-02 19:00:00,2132-01-02 19:00:00,32.313714


In [17]:
bolus_day = bolus_example[bolus_example.index.hour == 7]

In [19]:
bolus_day['Propofol'].sum()

112.43390759358539

In [25]:
type(bolus_day.index[0]) is pd.tslib.Timestamp

True

In [21]:
type(bolus_day.index[0])

pandas.tslib.Timestamp